<div style='float: right'><img src='pic/eisbahn.png'/></div>
## <div id='eisbahn' />アイスバーン

In [ ]:
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
from itertools import product
from pulp import *
from unionfind import unionfind
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
 . . . . . 
R . .*.*L .
 . . . . . 
. . . . . .
 . . U . . 
.*. .*. . .
 . . . . . 
. . . . . .
 . . . . . 
. R*.*. . R
 . . . . . """.split('\n')
nw, nh = len(data[0])//2, len(data)//2

### 問題
* INから入りOUTにいく１本の線をひきます
* 灰色のマスをアイスバーンとし、必ず通ります
* アイスバーンで曲がってはいけません
* アイスバーンのみ交差可です
* 矢印は必ず通ること

### 変数
* vh：0:L, 1:R (1)
* vv：0:U, 1:D (2)
* vhs (3)
* vvs (4)

### 制約
* vhsをvhで、vvsをvvで表現 (5)
* 矢印は必ず通ること (6)
* 各マスで入る数と出る数が同じこと (7)
* アイスバーンなら横は同じ、縦も同じこと（曲がらない） (8)
* アイスバーンなら線は2以上 (9)
* アイスバーンでないなら線は2以下 (10)
* 線は1つ (11)

In [ ]:
m = LpProblem()
vh = np.array(addbinvars(nh, nw+1, 2)) # 0:L, 1:R (1)
vv = np.array(addbinvars(nh+1, nw, 2)) # 0:U, 1:D (2)
vhs = np.array(addvars(nh, nw+1)) # (3)
vvs = np.array(addvars(nh+1, nw)) # (4)
for i, j in product(range(nh), range(nw+1)):
    c = data[i*2+1][j*2]
    m += lpDot([1,2], vh[i,j]) == vhs[i,j] # (5)
    m += lpSum(vhs[i,j]) <= 2 # (5)
    if c != '.' or j%nw == 0:
        m += vhs[i,j] == (1 if c=='L' else 2 if c=='R' else 0) # (6)
for i, j in product(range(nh+1), range(nw)):
    c = data[i*2][j*2+1]
    m += lpDot([1,2], vv[i,j]) == vvs[i,j] # (5)
    m += lpSum(vvs[i,j]) <= 2 # (5)
    if c != '.' or i%nh == 0:
        m += vvs[i,j] == (1 if c=='U' else 2 if c=='D' else 0) # (6)
for i, j in product(range(nh), range(nw)):
    e1 = lpSum(vv[i+k,j,1-k] + vh[i,j+k,1-k] for k in range(2)) # in
    e2 = lpSum(vv[i+k,j,k]   + vh[i,j+k,k] for k in range(2)) # out
    m += e1 == e2 # (7)
    if data[i*2+1][j*2+1] == '*':
        m += vhs[i,j] == vhs[i,j+1] # (8)
        m += vvs[i,j] == vvs[i+1,j] # (8)
        m += e1 + e2 >= 2 # (9)
    else:
        m += e1 + e2 <= 2 # (10)
while True:
    %time m.solve()
    rhs = np.vectorize(value)(vhs)
    rvs = np.vectorize(value)(vvs)
    if m.status != 1: break
    u = unionfind(nh * nw)
    for i, j in product(range(nh), range(nw)):
        if i and rvs[i,j]:
            u.unite(j+i*nw-nw, j+i*nw)
        if j and rhs[i,j]:
            u.unite(j-1+i*nw, j+i*nw)
    if sum(1 for gr in u.groups() if len(gr) > 1) == 1:
        break
    m += lpSum(vh[rhs>0])+lpSum(vv[rvs>0]) <= (rhs>0).sum() + (rvs>0).sum() - 1 # (11)
r = (rvs[:-1,:]>0)+2*(rhs[:,:-1]>0)+4*(rvs[1:,:]>0)+8*(rhs[:,1:]>0)
print('\n'.join(''.join(s) for s in np.vectorize(lambda x: '　12┘4│┐78└─1┌34┼'[x])(r)))